# Refactor Result

In [154]:
from pathlib import Path

res_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result")

In [155]:
import jsonlines

def _read_c2(dir: Path):
    big_table = []
    for eval_metric_dir in dir.iterdir():
        if eval_metric_dir.is_dir():
            for dataset_dir in eval_metric_dir.iterdir():
                if dataset_dir.is_dir():
                    for model_dir in dataset_dir.iterdir():
                        if model_dir.is_dir():
                            for tau_dir in model_dir.iterdir():
                                if tau_dir.is_dir():
                                    inf_file = tau_dir / "inference.jsonl"
                                    temp_table = []
                                    with jsonlines.open(inf_file) as reader:
                                        for obj in reader:
                                            obj["task"] = "c2"
                                            obj["model"] = model_dir.name
                                            obj["dataset"] = dataset_dir.name
                                            obj["eval_metric"] = eval_metric_dir.name
                                            obj["tau"] = tau_dir.name
                                            obj["is_trap"] = True if "w-trap" in dir.parent.name else False
                                            temp_table.append(obj)
                                    big_table.extend(temp_table)
    return big_table


def _read_c1(dir: Path):
    big_table = []
    for dataset_dir in dir.iterdir():
        if dataset_dir.is_dir():
            for model_dir in dataset_dir.iterdir():
                if model_dir.is_dir():
                    for prompt_level_dir in model_dir.iterdir():
                        if prompt_level_dir.is_dir():
                            for dof_dir in prompt_level_dir.iterdir():
                                if dof_dir.is_dir():
                                    inf_file = dof_dir / "inference.jsonl"
                                    with jsonlines.open(inf_file) as reader:
                                        for obj in reader:
                                            obj["task"] = "c1"
                                            obj["model"] = model_dir.name
                                            obj["dataset"] = dataset_dir.name
                                            obj["eval_metric"] = dof_dir.name
                                            obj["prompt_level"] = prompt_level_dir.name
                                            obj["is_trap"] = True if "w-trap" in dir.parent.name else False
                                            big_table.append(obj)
    return big_table


def _read_c1_cv(dir: Path):
    big_table = []
    for dataset_dir in dir.iterdir():
        if dataset_dir.is_dir():
            for model_dir in dataset_dir.iterdir():
                if model_dir.is_dir():
                    for dof_dir in model_dir.iterdir():
                        if dof_dir.is_dir():
                            inf_file = dof_dir / "inference.jsonl"
                            with jsonlines.open(inf_file) as reader:
                                for obj in reader:
                                    obj["task"] = "c1"
                                    obj["model"] = model_dir.name
                                    obj["dataset"] = dataset_dir.name
                                    obj["eval_metric"] = dof_dir.name
                                    big_table.append(obj)
    return big_table

def _read_c2_cv(dir: Path):
    big_table = []
    for dataset_dir in dir.iterdir():
        if dataset_dir.is_dir():
            for tau_dir in dataset_dir.iterdir():
                if tau_dir.is_dir():
                    for model_dir in tau_dir.iterdir():
                        if model_dir.is_dir():
                            for eval_metric_dir in model_dir.iterdir():
                                if eval_metric_dir.is_dir():
                                    inf_file = eval_metric_dir / "inference.jsonl"
                                    with jsonlines.open(inf_file) as reader:
                                        for obj in reader:
                                            obj["task"] = "c2"
                                            obj["model"] = model_dir.name
                                            obj["dataset"] = dataset_dir.name
                                            obj["eval_metric"] = eval_metric_dir.name
                                            obj["tau"] = tau_dir.name
                                            big_table.append(obj)
    return big_table

In [156]:
c1 = _read_c1(res_dir / "final-table-w-trap/single-dof-cls") + _read_c1(res_dir / "final-table-wo-trap/single-dof-cls") + _read_c1_cv(res_dir / "final-table-cv-methods/single-dof-cls")
c2 = _read_c2(res_dir / "final-table-w-trap/obj-centered-cls") + _read_c2(res_dir / "final-table-wo-trap/obj-centered-cls") + _read_c2_cv(res_dir / "final-table-cv-methods/obj-centered-cls")

In [157]:
import pandas as pd

big_table = pd.DataFrame(c1 + c2)

In [158]:
big_table.columns

Index(['scene', 'pair', 'tx', 'ty', 'tz', 'theta', 'phi', 'psi', 'tx_text',
       'ty_text', 'tz_text', 'theta_text', 'phi_text', 'psi_text',
       'significance', 'significance_text', 'significance_value', 'rsn',
       'pred', 'label', 'is_correct', 'is_parse', 'round', 'task', 'model',
       'dataset', 'eval_metric', 'prompt_level', 'is_trap', 'seq', 'hash_id',
       'pred_val', 'label_val', 'is_valid', 'distance', 'angle', 'tau'],
      dtype='object')

In [159]:
big_table.to_json(res_dir / "main_results.jsonl", orient="records", lines=True)

## Start Data Analysis!!!

The default setup:
- for c1: dataset + model + eval_dof -- acc. (default is prompt_level = zero-shot + is_trap = false)
- for c2: dataset + model + evel_dof + tau -- acc. (default is is_trap = false)

In [160]:
c1_df = pd.DataFrame(c1)
c2_df = pd.DataFrame(c2)

In [161]:
c1_df["model"].value_counts()

model
Qwen2.5-VL-72B-Instruct           8466
Qwen2.5-VL-32B-Instruct           7288
gpt-4o                            5408
llava-onevision-qwen2-7b-ov-hf    5408
SpaceQwen2.5-VL-3B-Instruct       5408
Idefics3-8B-Llama3                5408
Qwen2.5-VL-7B-Instruct            5408
loftr                              474
sift                               474
Name: count, dtype: int64

make nice table for final result

In [162]:
from sklearn.metrics import f1_score

# define some names
c1_nice_table = c1_df.copy()
model_list = [
    "sift",
    "loftr",
    "SpaceQwen2.5-VL-3B-Instruct",
    "Idefics3-8B-Llama3",
    "llava-onevision-qwen2-7b-ov-hf",
    "Qwen2.5-VL-7B-Instruct",
    "Qwen2.5-VL-32B-Instruct",
    "Qwen2.5-VL-72B-Instruct",
    "gpt-4o"
]
c1_nice_table["model"] = pd.Categorical(c1_nice_table["model"], categories=model_list, ordered=True)

# filter condition
c1_nice_table = c1_nice_table[(c1_nice_table["prompt_level"] == "zero-shot") | (c1_nice_table["prompt_level"].isna())] # want cv-method and zero-shot for VLM
c1_nice_table = c1_nice_table[c1_nice_table["is_trap"].isna() | (c1_nice_table["is_trap"] == False)] # wo trap is default

def compute_metrics(g):
    acc = g["is_correct"].mean()
    f1 = f1_score(g["label"], g["pred"], average="weighted")
    return pd.Series({"acc": acc, "f1": f1})

output = c1_nice_table.groupby(["dataset", "model", "eval_metric"], observed=True).apply(compute_metrics).pivot_table(index=["dataset", "model"], columns=["eval_metric"], values=["acc", "f1"], observed=True)

avg_output = c1_nice_table.groupby(["dataset", "model"], observed=True).apply(compute_metrics)
avg_output.columns = pd.MultiIndex.from_product([avg_output.columns.unique(), ["avg"]], names=[None, "eval_metric"])

# post-process output
output = output.round(2)
avg_output = avg_output.round(2)

/local/user/1483801110/ipykernel_218062/833211985.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  output = c1_nice_table.groupby(["dataset", "model", "eval_metric"], observed=True).apply(compute_metrics).pivot_table(index=["dataset", "model"], columns=["eval_metric"], values=["acc", "f1"], observed=True)
/local/user/1483801110/ipykernel_218062/833211985.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_ou

In [163]:
output = pd.concat([output, avg_output], axis=1).sort_index(axis=1)

In [164]:
output

acc                                \
eval_metric                                avg   phi   psi theta    tx    ty   
dataset   model                                                                
7-scenes  sift                            0.97  1.00  0.98  1.00  0.88  0.80   
          loftr                           0.98  0.98  0.98  1.00  0.97  0.80   
          SpaceQwen2.5-VL-3B-Instruct     0.53  0.55  0.48  0.43  0.66  0.50   
          Idefics3-8B-Llama3              0.52  0.52  0.40  0.50  0.66  0.60   
          llava-onevision-qwen2-7b-ov-hf  0.50  0.47  0.58  0.34  0.59  0.70   
          Qwen2.5-VL-7B-Instruct          0.57  0.55  0.45  0.66  0.59  0.70   
          Qwen2.5-VL-32B-Instruct         0.70  0.82  0.45  0.88  0.72  0.90   
          Qwen2.5-VL-72B-Instruct         0.77  0.82  0.58  0.84  0.88  0.90   
          gpt-4o                          0.76  0.90  0.45  0.84  0.88  1.00   
scannet   sift                            0.83  0.90  0.91  0.83  0.69  0.67   
          loftr                           0.94  1.00  0.97  0.97  0.85  0.67   
          SpaceQwen2.5-VL-3B-Instruct     0.44  0.42  0.38  0.45  0.42  0.47   
          Idefics3-8B-Llama3              0.51  0.53  0.50  0.52  0.50  0.47   
          llava-onevision-qwen2-7b-ov-hf  0.58  0.60  0.56  0.47  0.67  0.53   
          Qwen2.5-VL-7B-Instruct          0.72  0.68  0.56  0.90  0.71  0.73   
          Qwen2.5-VL-32B-Instruct         0.82  0.85  0.59  0.97  0.83  0.87   
          Qwen2.5-VL-72B-Instruct         0.82  0.92  0.29  0.97  0.89  0.84   
          gpt-4o                          0.79  0.88  0.38  1.00  0.75  1.00   
scannetpp SpaceQwen2.5-VL-3B-Instruct     0.55  0.50  0.50  0.60  0.50  0.50   
          Idefics3-8B-Llama3              0.46  0.45  0.50  0.38  0.53  0.00   
          llava-onevision-qwen2-7b-ov-hf  0.54  0.48  0.50  0.47  0.60  1.00   
          Qwen2.5-VL-7B-Instruct          0.72  0.60  0.50  0.92  0.63  0.50   
          Qwen2.5-VL-32B-Instruct         0.76  0.60  1.00  0.95  0.72  0.50   
          Qwen2.5-VL-72B-Instruct         0.90  0.93  0.00  0.96  0.85  0.88   
          gpt-4o                          0.88  0.83  0.50  0.97  0.85  1.00   

                                                  f1                          \
eval_metric                                 tz   avg   phi   psi theta    tx   
dataset   model                                                                
7-scenes  sift                            1.00  0.97  1.00  0.98  1.00  0.87   
          loftr                           1.00  0.98  0.98  0.98  1.00  0.97   
          SpaceQwen2.5-VL-3B-Instruct     0.67  0.53  0.55  0.47  0.45  0.66   
          Idefics3-8B-Llama3              0.72  0.53  0.52  0.40  0.52  0.66   
          llava-onevision-qwen2-7b-ov-hf  0.56  0.42  0.30  0.55  0.29  0.44   
          Qwen2.5-VL-7B-Instruct          0.67  0.56  0.50  0.50  0.67  0.49   
          Qwen2.5-VL-32B-Instruct         0.44  0.69  0.81  0.38  0.88  0.72   
          Qwen2.5-VL-72B-Instruct         0.72  0.79  0.82  0.64  0.88  0.89   
          gpt-4o                          0.78  0.77  0.90  0.45  0.85  0.88   
scannet   sift                            1.00  0.84  0.90  0.94  0.85  0.70   
          loftr                           1.00  0.94  1.00  0.97  0.97  0.87   
          SpaceQwen2.5-VL-3B-Instruct     0.58  0.44  0.42  0.39  0.46  0.43   
          Idefics3-8B-Llama3              0.47  0.51  0.53  0.50  0.51  0.51   
          llava-onevision-qwen2-7b-ov-hf  0.74  0.49  0.48  0.56  0.40  0.53   
          Qwen2.5-VL-7B-Instruct          0.63  0.71  0.69  0.59  0.91  0.64   
          Qwen2.5-VL-32B-Instruct         0.58  0.81  0.85  0.52  0.97  0.82   
          Qwen2.5-VL-72B-Instruct         0.59  0.83  0.93  0.36  0.97  0.89   
          gpt-4o                          0.53  0.79  0.88  0.39  1.00  0.76   
scannetpp SpaceQwen2.5-VL-3B-Instruct     0.75  0.55  0.50  0.67  0.60  0.50   
          Idefics3-8B-Llama3              

In [165]:
print(output.to_latex(
    column_format="ll" + "c" * len(output.columns),
    float_format="%.2f",
    label="tab:c1_results",
    caption="Results of C1 task with different models and evaluation metrics.",
))

\begin{table}
\caption{Results of C1 task with different models and evaluation metrics.}
\label{tab:c1_results}
\begin{tabular}{llcccccccccccccc}
\toprule
 &  & \multicolumn{7}{r}{acc} & \multicolumn{7}{r}{f1} \\
 & eval_metric & avg & phi & psi & theta & tx & ty & tz & avg & phi & psi & theta & tx & ty & tz \\
dataset & model &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{7-scenes} & sift & 0.97 & 1.00 & 0.98 & 1.00 & 0.88 & 0.80 & 1.00 & 0.97 & 1.00 & 0.98 & 1.00 & 0.87 & 0.84 & 1.00 \\
 & loftr & 0.98 & 0.98 & 0.98 & 1.00 & 0.97 & 0.80 & 1.00 & 0.98 & 0.98 & 0.98 & 1.00 & 0.97 & 0.80 & 1.00 \\
 & SpaceQwen2.5-VL-3B-Instruct & 0.53 & 0.55 & 0.48 & 0.43 & 0.66 & 0.50 & 0.67 & 0.53 & 0.55 & 0.47 & 0.45 & 0.66 & 0.52 & 0.67 \\
 & Idefics3-8B-Llama3 & 0.52 & 0.52 & 0.40 & 0.50 & 0.66 & 0.60 & 0.72 & 0.53 & 0.52 & 0.40 & 0.52 & 0.66 & 0.60 & 0.74 \\
 & llava-onevision-qwen2-7b-ov-hf & 0.50 & 0.47 & 0.58 & 0.34 & 0.59 & 0.70 & 0.56 & 0.42 & 0.30 & 0.55 & 0.29 & 0.

same for C2

In [166]:
c2_nice_table = c2_df.copy()
c2_nice_table = c2_nice_table[c2_nice_table["is_trap"].isna() | (c2_nice_table["is_trap"] == False)]
c2_nice_table["model"] = pd.Categorical(c2_nice_table["model"], categories=model_list, ordered=True)

output = c2_nice_table.groupby(["dataset", "model", "eval_metric", "tau"], observed=True).apply(compute_metrics).pivot_table(index=["dataset", "model"], columns=["eval_metric", "tau"], values=["acc", "f1"], observed=True)
output.columns = pd.MultiIndex.from_product([["acc", "f1"], ["phi", "tx"], ["15", "30", "45", "60"]], names=[None, "dof", "tau"])

avg_output = c2_nice_table.groupby(["dataset", "model", "eval_metric"], observed=True).apply(compute_metrics).pivot_table(index=["dataset", "model"], columns=["eval_metric"], values=["acc", "f1"], observed=True)
avg_output.columns = pd.MultiIndex.from_product([["acc", "f1"], ["phi", "tx"], ["avg"]], names=[None, "dof", "tau"])

output = output.round(2)

avg_output = avg_output.round(2)

output = pd.concat([output, avg_output], axis=1).sort_index(axis=1)

/local/user/1483801110/ipykernel_218062/1773341369.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  output = c2_nice_table.groupby(["dataset", "model", "eval_metric", "tau"], observed=True).apply(compute_metrics).pivot_table(index=["dataset", "model"], columns=["eval_metric", "tau"], values=["acc", "f1"], observed=True)
/local/user/1483801110/ipykernel_218062/1773341369.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this war

In [167]:
output

acc                                \
dof                                       phi                            tx   
tau                                        15    30    45    60   avg    15   
dataset  model                                                                
7-scenes sift                            0.59  0.44  0.34  0.30  0.43  0.50   
         loftr                           0.95  0.95  0.92  0.76  0.91  0.95   
         SpaceQwen2.5-VL-3B-Instruct     0.63  0.60  0.60  0.61  0.61  0.40   
         Idefics3-8B-Llama3              0.60  0.57  0.55  0.55  0.57  0.50   
         llava-onevision-qwen2-7b-ov-hf  0.55  0.52  0.57  0.58  0.55  0.40   
         Qwen2.5-VL-7B-Instruct          0.45  0.38  0.43  0.55  0.44  0.57   
         Qwen2.5-VL-32B-Instruct         0.45  0.45  0.47  0.50  0.46  0.67   
         Qwen2.5-VL-72B-Instruct         0.58  0.48  0.45  0.47  0.50  0.62   
         gpt-4o                          0.62  0.37  0.60  0.50  0.52  0.65   
scannet  sift                            0.18  0.17  0.15  0.11  0.15  0.22   
         loftr                           0.95  0.77  0.69  0.55  0.75  0.93   
         SpaceQwen2.5-VL-3B-Instruct     0.63  0.55  0.57  0.57  0.58  0.43   
         Idefics3-8B-Llama3              0.52  0.47  0.45  0.45  0.47  0.48   
         llava-onevision-qwen2-7b-ov-hf  0.55  0.45  0.45  0.53  0.50  0.43   
         Qwen2.5-VL-7B-Instruct          0.33  0.45  0.53  0.40  0.42  0.63   
         Qwen2.5-VL-32B-Instruct         0.42  0.38  0.38  0.47  0.41  0.52   
         Qwen2.5-VL-72B-Instruct         0.40  0.40  0.43  0.45  0.42  0.60   
         gpt-4o                          0.42  0.53  0.47  0.52  0.49  0.58   

                                                                   f1        \
dof                                                               phi         
tau                                        30    45    60   avg    15    30   
dataset  model                                                                
7-scenes sift                            0.42  0.38  0.21  0.39  0.72  0.56   
         loftr                           0.95  0.90  0.79  0.91  0.97  0.96   
         SpaceQwen2.5-VL-3B-Instruct     0.47  0.43  0.45  0.44  0.62  0.60   
         Idefics3-8B-Llama3              0.45  0.48  0.42  0.47  0.60  0.57   
         llava-onevision-qwen2-7b-ov-hf  0.48  0.43  0.42  0.44  0.55  0.52   
         Qwen2.5-VL-7B-Instruct          0.55  0.58  0.45  0.55  0.32  0.26   
         Qwen2.5-VL-32B-Instruct         0.57  0.47  0.34  0.53  0.44  0.46   
         Qwen2.5-VL-72B-Instruct         0.50  0.53  0.42  0.53  0.57  0.48   
         gpt-4o                          0.58  0.45  0.34  0.52  0.62  0.37   
scannet  sift                            0.20  0.12  0.15  0.18  0.28  0.27   
         loftr                           0.77  0.78  0.53  0.75  0.96  0.81   
         SpaceQwen2.5-VL-3B-Instruct     0.53  0.55  0.48  0.50  0.63  0.54   
         Idefics3-8B-Llama3              0.53  0.53  0.40  0.49  0.53  0.47   
         llava-onevision-qwen2-7b-ov-hf  0.53  0.53  0.47  0.49  0.57  0.46   
         Qwen2.5-VL-7B-Instruct          0.52  0.42  0.58  0.55  0.23  0.37   
         Qwen2.5-VL-32B-Instruct         0.58  0.53  0.53  0.54  0.42  0.37   
         Qwen2.5-VL-72B-Instruct         0.62  0.62  0.53  0.59  0.38  0.40   
         gpt-4o                          0.48  0.47  0.48  0.51  0.44  0.52   

                                                                             \
dof                                                          tx               
tau                                        45    60   avg    15    30    45   
dataset  model                                                                
7-scenes sift                            0.46  0.42  0.56  0.61  0.53  0.50   
         loftr                           0.95  0.82  0.93  0.97  0.96  0.93   
         SpaceQwen2.5-VL-3B-Instruct     0.60  0.59  0.60  0.41  0.47  0.44   
         Id

In [168]:
print(output.to_latex(
    column_format="ll" + "c" * len(output.columns),
    float_format="%.2f",
    label="tab:c2_results",
    caption="Results of C2 task with different models, evaluation metrics, and tau values.",
))

\begin{table}
\caption{Results of C2 task with different models, evaluation metrics, and tau values.}
\label{tab:c2_results}
\begin{tabular}{llcccccccccccccccccccc}
\toprule
 &  & \multicolumn{10}{r}{acc} & \multicolumn{10}{r}{f1} \\
 & dof & \multicolumn{5}{r}{phi} & \multicolumn{5}{r}{tx} & \multicolumn{5}{r}{phi} & \multicolumn{5}{r}{tx} \\
 & tau & 15 & 30 & 45 & 60 & avg & 15 & 30 & 45 & 60 & avg & 15 & 30 & 45 & 60 & avg & 15 & 30 & 45 & 60 & avg \\
dataset & model &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{7-scenes} & sift & 0.59 & 0.44 & 0.34 & 0.30 & 0.43 & 0.50 & 0.42 & 0.38 & 0.21 & 0.39 & 0.72 & 0.56 & 0.46 & 0.42 & 0.56 & 0.61 & 0.53 & 0.50 & 0.29 & 0.51 \\
 & loftr & 0.95 & 0.95 & 0.92 & 0.76 & 0.91 & 0.95 & 0.95 & 0.90 & 0.79 & 0.91 & 0.97 & 0.96 & 0.95 & 0.82 & 0.93 & 0.97 & 0.96 & 0.93 & 0.85 & 0.93 \\
 & SpaceQwen2.5-VL-3B-Instruct & 0.63 & 0.60 & 0.60 & 0.61 & 0.61 & 0.40 & 0.47 & 0.43 & 0.45 & 0.44 & 0.62 & 0.60 & 0.60

## How about Trap Option?

Since both C1 and C2 dataset have 2 setup, but we combinate this time.

In [169]:
trap_table = big_table.copy()

trap_table = trap_table[~trap_table["model"].isin(["sift", "loftr"])]
trap_table = trap_table[(trap_table["prompt_level"] == "zero-shot") | (trap_table["prompt_level"].isna())]

model_list_trap = [model for model in model_list if model not in ["sift", "loftr"]]
trap_table["model"] = pd.Categorical(trap_table["model"], categories=model_list_trap, ordered=True)
trap_table["is_trap"] = pd.Categorical(trap_table["is_trap"], categories=[False, True], ordered=True)

In [170]:
trap_table = trap_table.groupby(["task", "model", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["task", "model"], columns="is_trap", values=["acc", "f1"])
trap_table

/local/user/1483801110/ipykernel_218062/1041263776.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trap_table = trap_table.groupby(["task", "model", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["task", "model"], columns="is_trap", values=["acc", "f1"])
/local/user/1483801110/ipykernel_218062/1041263776.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trap_table = trap_table.groupby(["task", "model", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["task", "model"], columns="is

acc                  f1          
is_trap                                 False      True     False      True
task model                                                                 
c1   SpaceQwen2.5-VL-3B-Instruct     0.501479  0.071006  0.503007  0.121841
     Idefics3-8B-Llama3              0.498521  0.250000  0.502368  0.342608
     llava-onevision-qwen2-7b-ov-hf  0.541420  0.110947  0.452324  0.165449
     Qwen2.5-VL-7B-Instruct          0.670118  0.215976  0.648192  0.299978
     Qwen2.5-VL-32B-Instruct         0.774123  0.448718  0.765405  0.561254
     Qwen2.5-VL-72B-Instruct         0.834718  0.783956  0.849372  0.827607
     gpt-4o                          0.807692  0.739645  0.809948  0.748586
c2   SpaceQwen2.5-VL-3B-Instruct     0.531659  0.253275  0.529887  0.333910
     Idefics3-8B-Llama3              0.497817  0.359170  0.500809  0.423724
     llava-onevision-qwen2-7b-ov-hf  0.493450  0.272926  0.498041  0.331924
     Qwen2.5-VL-7B-Instruct          0.489631  0.403226  0.397714  0.404246
     Qwen2.5-VL-32B-Instruct         0.485808  0.321701  0.477512  0.376240
     Qwen2.5-VL-72B-Instruct         0.509825  0.497817  0.504708  0.494589
     gpt-4o                          0.510369  0.523041  0.512641  0.521844

In [171]:
print(trap_table.to_latex(
    float_format="%.2f",
))

\begin{tabular}{llrrrr}
\toprule
 &  & \multicolumn{2}{r}{acc} & \multicolumn{2}{r}{f1} \\
 & is_trap & False & True & False & True \\
task & model &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{c1} & SpaceQwen2.5-VL-3B-Instruct & 0.50 & 0.07 & 0.50 & 0.12 \\
 & Idefics3-8B-Llama3 & 0.50 & 0.25 & 0.50 & 0.34 \\
 & llava-onevision-qwen2-7b-ov-hf & 0.54 & 0.11 & 0.45 & 0.17 \\
 & Qwen2.5-VL-7B-Instruct & 0.67 & 0.22 & 0.65 & 0.30 \\
 & Qwen2.5-VL-32B-Instruct & 0.77 & 0.45 & 0.77 & 0.56 \\
 & Qwen2.5-VL-72B-Instruct & 0.83 & 0.78 & 0.85 & 0.83 \\
 & gpt-4o & 0.81 & 0.74 & 0.81 & 0.75 \\
\cline{1-6}
\multirow[t]{7}{*}{c2} & SpaceQwen2.5-VL-3B-Instruct & 0.53 & 0.25 & 0.53 & 0.33 \\
 & Idefics3-8B-Llama3 & 0.50 & 0.36 & 0.50 & 0.42 \\
 & llava-onevision-qwen2-7b-ov-hf & 0.49 & 0.27 & 0.50 & 0.33 \\
 & Qwen2.5-VL-7B-Instruct & 0.49 & 0.40 & 0.40 & 0.40 \\
 & Qwen2.5-VL-32B-Instruct & 0.49 & 0.32 & 0.48 & 0.38 \\
 & Qwen2.5-VL-72B-Instruct & 0.51 & 0.50 & 0.50 & 0.49 \\
 & gpt-4o & 0.51 & 0.52 &

## How about Prompt Strategy

In [172]:
prompt_table = c1_df.copy()

prompt_table = prompt_table[~prompt_table["model"].isin(["sift", "loftr"])]
prompt_table["model"] = pd.Categorical(prompt_table["model"], categories=model_list_trap, ordered=True)
prompt_table["prompt_level"] = pd.Categorical(prompt_table["prompt_level"], categories=[
    "zero-shot", 
    "dataset-prior-hint",
    "CoT-hint",
    "VoT-hint",
], ordered=True)

In [173]:
prompt_table = prompt_table.groupby(["model", "prompt_level"]).apply(compute_metrics).pivot_table(index="model", columns="prompt_level", values=["acc", "f1"], observed=True)
prompt_table

/local/user/1483801110/ipykernel_218062/1312274158.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  prompt_table = prompt_table.groupby(["model", "prompt_level"]).apply(compute_metrics).pivot_table(index="model", columns="prompt_level", values=["acc", "f1"], observed=True)
/local/user/1483801110/ipykernel_218062/1312274158.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prompt_table = prompt_table.groupby(["model", "prompt_level"]).apply(compute_metrics).pivot_table(index="model", columns="prompt_level", v

acc                               \
prompt_level                   zero-shot dataset-prior-hint  CoT-hint   
model                                                                   
SpaceQwen2.5-VL-3B-Instruct     0.286243           0.392012  0.409024   
Idefics3-8B-Llama3              0.374260           0.474112  0.504438   
llava-onevision-qwen2-7b-ov-hf  0.326183           0.464497  0.438609   
Qwen2.5-VL-7B-Instruct          0.443047           0.602811  0.457840   
Qwen2.5-VL-32B-Instruct         0.596806           0.738487  0.644904   
Qwen2.5-VL-72B-Instruct         0.808770           0.801184  0.589391   
gpt-4o                          0.773669           0.743343  0.551775   

                                                f1                     \
prompt_level                    VoT-hint zero-shot dataset-prior-hint   
model                                                                   
SpaceQwen2.5-VL-3B-Instruct     0.422337  0.365161           0.430792   
Idefics3-8B-Llama3              0.469675  0.436002           0.488488   
llava-onevision-qwen2-7b-ov-hf  0.468935  0.351719           0.440272   
Qwen2.5-VL-7B-Instruct          0.477071  0.512270           0.579210   
Qwen2.5-VL-32B-Instruct         0.690355  0.670075           0.752911   
Qwen2.5-VL-72B-Instruct         0.654124  0.839160           0.803365   
gpt-4o                          0.486686  0.779820           0.738757   

                                                    
prompt_level                    CoT-hint  VoT-hint  
model                                               
SpaceQwen2.5-VL-3B-Instruct     0.434801  0.441852  
Idefics3-8B-Llama3              0.512551  0.450233  
llava-onevision-qwen2-7b-ov-hf  0.443518  0.431512  
Qwen2.5-VL-7B-Instruct          0.384622  0.436887  
Qwen2.5-VL-32B-Instruct         0.664250  0.710102  
Qwen2.5-VL-72B-Instruct         0.574388  0.649181  
gpt-4o                          0.539478  0.472663

In [174]:
print(prompt_table.to_latex(
    float_format="%.2f",
))

\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{4}{r}{acc} & \multicolumn{4}{r}{f1} \\
prompt_level & zero-shot & dataset-prior-hint & CoT-hint & VoT-hint & zero-shot & dataset-prior-hint & CoT-hint & VoT-hint \\
model &  &  &  &  &  &  &  &  \\
\midrule
SpaceQwen2.5-VL-3B-Instruct & 0.29 & 0.39 & 0.41 & 0.42 & 0.37 & 0.43 & 0.43 & 0.44 \\
Idefics3-8B-Llama3 & 0.37 & 0.47 & 0.50 & 0.47 & 0.44 & 0.49 & 0.51 & 0.45 \\
llava-onevision-qwen2-7b-ov-hf & 0.33 & 0.46 & 0.44 & 0.47 & 0.35 & 0.44 & 0.44 & 0.43 \\
Qwen2.5-VL-7B-Instruct & 0.44 & 0.60 & 0.46 & 0.48 & 0.51 & 0.58 & 0.38 & 0.44 \\
Qwen2.5-VL-32B-Instruct & 0.60 & 0.74 & 0.64 & 0.69 & 0.67 & 0.75 & 0.66 & 0.71 \\
Qwen2.5-VL-72B-Instruct & 0.81 & 0.80 & 0.59 & 0.65 & 0.84 & 0.80 & 0.57 & 0.65 \\
gpt-4o & 0.77 & 0.74 & 0.55 & 0.49 & 0.78 & 0.74 & 0.54 & 0.47 \\
\bottomrule
\end{tabular}



## Let's See a Combination

In [175]:
trap_table = c1_df.copy()

trap_table = trap_table[~trap_table["model"].isin(["sift", "loftr"])]
# trap_table = trap_table[trap_table["prompt_level"] == "zero-shot"]
model_list_trap = [model for model in model_list if model not in ["sift", "loftr"]]
trap_table["model"] = pd.Categorical(trap_table["model"], categories=model_list_trap, ordered=True)
trap_table["prompt_level"] = pd.Categorical(trap_table["prompt_level"], categories=[
    "zero-shot", 
    "dataset-prior-hint",
    "CoT-hint",
    "VoT-hint",
], ordered=True)
trap_table["is_trap"] = pd.Categorical(trap_table["is_trap"], categories=[True, False], ordered=True)

In [176]:
trap_table = trap_table.groupby(["model", "prompt_level", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["model", "is_trap"], columns=["prompt_level"], values=["acc", "f1"])
trap_table

/local/user/1483801110/ipykernel_218062/2832700779.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trap_table = trap_table.groupby(["model", "prompt_level", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["model", "is_trap"], columns=["prompt_level"], values=["acc", "f1"])
/local/user/1483801110/ipykernel_218062/2832700779.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trap_table = trap_table.groupby(["model", "prompt_level", "is_trap"]).apply(compute_metrics).reset_index().pivot(index=["mo

acc                               \
prompt_level                           zero-shot dataset-prior-hint  CoT-hint   
model                          is_trap                                          
SpaceQwen2.5-VL-3B-Instruct    True     0.071006           0.288462  0.315089   
                               False    0.501479           0.495562  0.502959   
Idefics3-8B-Llama3             True     0.250000           0.439349  0.473373   
                               False    0.498521           0.508876  0.535503   
llava-onevision-qwen2-7b-ov-hf True     0.110947           0.439349  0.396450   
                               False    0.541420           0.489645  0.480769   
Qwen2.5-VL-7B-Instruct         True     0.215976           0.560651  0.452663   
                               False    0.670118           0.644970  0.463018   
Qwen2.5-VL-32B-Instruct        True     0.448718           0.709430  0.619342   
                               False    0.774123           0.767544  0.672149   
Qwen2.5-VL-72B-Instruct        True     0.783956           0.794872  0.613553   
                               False    0.834718           0.808556  0.561441   
gpt-4o                         True     0.739645           0.726331  0.526627   
                               False    0.807692           0.760355  0.576923   

                                                        f1                     \
prompt_level                            VoT-hint zero-shot dataset-prior-hint   
model                          is_trap                                          
SpaceQwen2.5-VL-3B-Instruct    True     0.319527  0.121841           0.366036   
                               False    0.525148  0.503007           0.475364   
Idefics3-8B-Llama3             True     0.471893  0.342608           0.468821   
                               False    0.467456  0.502368           0.504518   
llava-onevision-qwen2-7b-ov-hf True     0.448225  0.165449           0.448105   
                               False    0.489645  0.452324           0.423257   
Qwen2.5-VL-7B-Instruct         True     0.465976  0.299978           0.539490   
                               False    0.488166  0.648192           0.614073   
Qwen2.5-VL-32B-Instruct        True     0.637516  0.561254           0.741330   
                               False    0.743329  0.765405           0.762493   
Qwen2.5-VL-72B-Instruct        True     0.668322  0.827607           0.802484   
                               False    0.638800  0.849372           0.803167   
gpt-4o                         True     0.476331  0.748586           0.718515   
                               False    0.497041  0.809948           0.757918   

                                                            
prompt_level                            CoT-hint  VoT-hint  
model                          is_trap                      
SpaceQwen2.5-VL-3B-Instruct    True     0.373071  0.376624  
                               False    0.481015  0.492853  
Idefics3-8B-Llama3             True     0.489697  0.449636  
                               False    0.532508  0.445257  
llava-onevision-qwen2-7b-ov-hf True     0.422171  0.445837  
                               False    0.437251  0.408460  
Qwen2.5-VL-7B-Instruct         True     0.389269  0.415296  
                               False    0.374323  0.445790  
Qwen2.5-VL-32B-Instruct        True     0.653401  0.674561  
                               False    0.670409  0.741434  
Qwen2.5-VL-72B-Instruct        True     0.608908  0.665094  
                               False    0.527600  0.630358  
gpt-4o                         True     0.520164  0.461981  
                               False    0.551408  0.482850

In [177]:
print(trap_table.to_latex(
    float_format="%.2f",
))

\begin{tabular}{llrrrrrrrr}
\toprule
 &  & \multicolumn{4}{r}{acc} & \multicolumn{4}{r}{f1} \\
 & prompt_level & zero-shot & dataset-prior-hint & CoT-hint & VoT-hint & zero-shot & dataset-prior-hint & CoT-hint & VoT-hint \\
model & is_trap &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{SpaceQwen2.5-VL-3B-Instruct} & True & 0.07 & 0.29 & 0.32 & 0.32 & 0.12 & 0.37 & 0.37 & 0.38 \\
 & False & 0.50 & 0.50 & 0.50 & 0.53 & 0.50 & 0.48 & 0.48 & 0.49 \\
\cline{1-10}
\multirow[t]{2}{*}{Idefics3-8B-Llama3} & True & 0.25 & 0.44 & 0.47 & 0.47 & 0.34 & 0.47 & 0.49 & 0.45 \\
 & False & 0.50 & 0.51 & 0.54 & 0.47 & 0.50 & 0.50 & 0.53 & 0.45 \\
\cline{1-10}
\multirow[t]{2}{*}{llava-onevision-qwen2-7b-ov-hf} & True & 0.11 & 0.44 & 0.40 & 0.45 & 0.17 & 0.45 & 0.42 & 0.45 \\
 & False & 0.54 & 0.49 & 0.48 & 0.49 & 0.45 & 0.42 & 0.44 & 0.41 \\
\cline{1-10}
\multirow[t]{2}{*}{Qwen2.5-VL-7B-Instruct} & True & 0.22 & 0.56 & 0.45 & 0.47 & 0.30 & 0.54 & 0.39 & 0.42 \\
 & False & 0.67 & 0.64 & 0.46 & 0.49